In [129]:
from os import getcwd, listdir, path

cwd = getcwd()
input_dir = path.join(cwd, '../data/prepared')
input_files = [ path.join(input_dir, file) for file in listdir(input_dir) if not file.endswith('05.json')]
input_files

['/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-04.json',
 '/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-03.json',
 '/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-02.json',
 '/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-01.json']

In [130]:
import pandas as pd
translation_df = pd.read_json(path.join(cwd, '../data/label_lookup.json'))
translation_df

,entity,nasdaq_label,nyse_label
3,mer telemanagement solutions ltd.,"[MTSL, mer telemanagement solutions ltd, 0.0]",None
27,"national retail properties, inc",None,"[NNN, national retail properties, 0.0]"
31,louisiana-pacific,None,"[LPX, louisiana-pacific corp, 0.0]"
35,qualys inc,"[QLYS, qualys inc, 0.0]",None
36,chaparral energy inc,None,"[CHAP, chaparral energy inc, 0.0]"
...,...,...,...
144167,oclaro inc.,"[OCLR, oclaro inc, 0.0]",None
144189,proteon therapeutics inc,"[NTGN, neon therapeutics inc, 0.0476190476]",None
144222,cfm international inc.,None,"[RPM, rpm international inc, 0.0952380952]"
144227,relx plc,None,"[RELX, relx plc, 0.0]"


In [131]:
# All entity values are unique
translation_df['entity'].unique().shape[0] / translation_df.shape[0]

1.0

In [132]:
translation_df = translation_df.set_index('entity')

In [133]:
translation_df

,nasdaq_label,nyse_label
entity,,
mer telemanagement solutions ltd.,"[MTSL, mer telemanagement solutions ltd, 0.0]",None
"national retail properties, inc",None,"[NNN, national retail properties, 0.0]"
louisiana-pacific,None,"[LPX, louisiana-pacific corp, 0.0]"
qualys inc,"[QLYS, qualys inc, 0.0]",None
chaparral energy inc,None,"[CHAP, chaparral energy inc, 0.0]"
...,...,...
oclaro inc.,"[OCLR, oclaro inc, 0.0]",None
proteon therapeutics inc,"[NTGN, neon therapeutics inc, 0.0476190476]",None
cfm international inc.,None,"[RPM, rpm international inc, 0.0952380952]"


In [139]:
def remove_published(row):
    row.loc['entities.organizations'] = list(filter(
        lambda d: d['name'].strip() != row.loc['entities_blacklist'] and d['name'].strip() != 'thomson reuters',
        row.loc['entities.organizations']
    ))
    return row
df = pd.read_json(input_files[0])
df['entities_blacklist'] = df['thread.site_full'].apply(lambda entry: entry.split('.')[-2])
df = df.apply(remove_published, axis=1)
df['entities.organizations'] = df['entities.organizations'].apply(
    lambda l: list(map(lambda item: item['name'], l))
)
df = df.drop(columns=['thread.site_full','entities_blacklist'])
df

,title,entities.organizations,text,published
0,Mo Salah wins PFA player of the year award,"[pfa, reuters news agency | brand attribution ...",Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00
1,Motor racing-Mercedes look to China after losi...,[mercedes],"April 9, 2018 / 3:17 PM / a few seconds ago Mo...",2018-04-09T17:52:00.000+03:00
2,URGENT-Hedge fund Elliott issues new attack on...,"[telecom italia, vivendi]","PARIS, April 19 (Reuters) - Activist Elliott A...",2018-04-19T14:17:00.000+03:00
3,U.S. FAA to order inspections of jet engines a...,"[southwest, faa, ntsb, min read, southwest air...",(Reuters) - The U.S. Federal Aviation Administ...,2018-04-19T17:56:00.000+03:00
4,"Givaudan Q1 sales rise 5.4 pct, helped by new ...",[givaudan],"ZURICH, April 10 (Reuters) - Fragrance and fla...",2018-04-10T13:06:00.000+03:00
...,...,...,...,...
63240,- Comcast is best leaving Fox to the Mouse,"[comcast, sky, disneyland paris]",NEW YORK (Reuters Breakingviews) - Comcast bos...,2018-04-26T04:00:00.000+03:00
63241,Hasbro blames Toys 'R' Us for dismal first qua...,[hasbro],Hasbro blames Toys 'R' Us for dismal first qua...,2018-04-23T22:40:00.000+03:00
63242,BRIEF-IHH Healthcare Is Said To Propose Up To ...,"[fortis, bloomberg, tpg, g - bloomberg]",April 12 (Reuters) -\n* IHH HEALTHCARE IS SAID...,2018-04-12T13:50:00.000+03:00
63243,Rusal non-executive directors exit board after...,"[united company rusal plc, u.s. department of ...",(Reuters) - Russian aluminum producer United C...,2018-04-11T07:06:00.000+03:00


In [140]:
transformed_df = pd.DataFrame(
    [(row[1], entity, row[3], row[4]) for row in df.itertuples() for entity in row[2]],
    columns = df.columns
).rename(columns={'entities.organizations': 'entity'})
transformed_df.head()

,title,entity,text,published
0,Mo Salah wins PFA player of the year award,pfa,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00
1,Mo Salah wins PFA player of the year award,reuters news agency | brand attribution guidel...,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00
2,Mo Salah wins PFA player of the year award,reuters.com,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00
3,Mo Salah wins PFA player of the year award,reuters tv,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00
4,Motor racing-Mercedes look to China after losi...,mercedes,"April 9, 2018 / 3:17 PM / a few seconds ago Mo...",2018-04-09T17:52:00.000+03:00


In [141]:
transformed_df['nasdaq_label'] = transformed_df['entity'].apply(
    lambda e: translation_df.loc[e,'nasdaq_label'] if e in translation_df.index else None
)
transformed_df['nyse_label'] = transformed_df['entity'].apply(
    lambda e: translation_df.loc[e,'nyse_label'] if e in translation_df.index else None
)
transformed_df.head()

,title,entity,text,published,nasdaq_label,nyse_label
0,Mo Salah wins PFA player of the year award,pfa,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00,None,None
1,Mo Salah wins PFA player of the year award,reuters news agency | brand attribution guidel...,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00,None,None
2,Mo Salah wins PFA player of the year award,reuters.com,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00,None,None
3,Mo Salah wins PFA player of the year award,reuters tv,Mo Salah wins PFA player of the year award 10:...,2018-04-23T12:35:00.000+03:00,None,None
4,Motor racing-Mercedes look to China after losi...,mercedes,"April 9, 2018 / 3:17 PM / a few seconds ago Mo...",2018-04-09T17:52:00.000+03:00,None,None


In [143]:
# 93% percent of articles has no related label found
transformed_df[
    (transformed_df['nasdaq_label'].isna()) & (transformed_df['nyse_label']).isna()
].shape[0] / transformed_df.shape[0]

0.9345857607296372

In [144]:
transformed_df = transformed_df[
    (transformed_df['nasdaq_label'].notna()) | (transformed_df['nyse_label']).notna()
]
transformed_df

,title,entity,text,published,nasdaq_label,nyse_label
49,West Virginia sues Equifax over data breach,equifax inc,April 12 (Reuters) - West Virginia’s attorney ...,2018-04-13T03:29:00.000+03:00,None,"[EFX, equifax inc, 0.0]"
81,BRIEF-Emerald Expositions Says Board Elected L...,emerald expositions events inc,April 6 (Reuters) - Emerald Expositions Events...,2018-04-07T04:41:00.000+03:00,None,"[EEX, emerald expositions events inc, 0.0]"
108,New Residential Announces First Quarter 2018 R...,new residential investment corp.,NEW YORK--(BUSINESS WIRE)-- New Residential In...,2018-04-27T14:58:00.000+03:00,None,"[NRZ, new residential investment corp, 0.0]"
128,Bill Ackman’s Pershing Square Faces Wave of In...,blackstone group lp,"William Ackman, the famed shareholder activist...",2018-04-05T13:30:00.000+03:00,None,"[BX, the blackstone group lp, 0.0]"
130,Bill Ackman’s Pershing Square Faces Wave of In...,jpmorgan chase & co.,"William Ackman, the famed shareholder activist...",2018-04-05T13:30:00.000+03:00,None,"[JPM, jp morgan chase & co, 0.1]"
...,...,...,...,...,...,...
183236,"Southwest not inspecting all CFM engines, sour...",southwest airlines co,NEW YORK (Reuters) - Southwest Airlines Co is ...,2018-04-19T01:22:00.000+03:00,None,"[LUV, southwest airlines company, 0.0]"
183253,The Rock’s Rampage Storms to No. 1 at Box Office,comscore inc.,The Rock is at it again.\nThe action-movie sta...,2018-04-15T21:48:00.000+03:00,"[SCOR, comscore inc, 0.0]",None
183291,Proxy adviser ISS backs call for gun safety re...,sturm ruger & co,"April 25, 2018 / 9:17 PM / in a day Proxy advi...",2018-04-26T00:16:00.000+03:00,None,"[RGR, sturm ruger & company, 0.0]"
183302,HSBC names veteran banker head of Asia belt an...,hsbc holdings plc,HONG KONG (Reuters) - HSBC Holdings Plc said o...,2018-04-09T14:51:00.000+03:00,None,"[HSBC, hsbc holdings plc, 0.0]"


In [145]:
import dateutil
transformed_df['published'] = transformed_df['published'].apply(pd.Timestamp)
transformed_df

,title,entity,text,published,nasdaq_label,nyse_label
49,West Virginia sues Equifax over data breach,equifax inc,April 12 (Reuters) - West Virginia’s attorney ...,2018-04-13 03:29:00+03:00,None,"[EFX, equifax inc, 0.0]"
81,BRIEF-Emerald Expositions Says Board Elected L...,emerald expositions events inc,April 6 (Reuters) - Emerald Expositions Events...,2018-04-07 04:41:00+03:00,None,"[EEX, emerald expositions events inc, 0.0]"
108,New Residential Announces First Quarter 2018 R...,new residential investment corp.,NEW YORK--(BUSINESS WIRE)-- New Residential In...,2018-04-27 14:58:00+03:00,None,"[NRZ, new residential investment corp, 0.0]"
128,Bill Ackman’s Pershing Square Faces Wave of In...,blackstone group lp,"William Ackman, the famed shareholder activist...",2018-04-05 13:30:00+03:00,None,"[BX, the blackstone group lp, 0.0]"
130,Bill Ackman’s Pershing Square Faces Wave of In...,jpmorgan chase & co.,"William Ackman, the famed shareholder activist...",2018-04-05 13:30:00+03:00,None,"[JPM, jp morgan chase & co, 0.1]"
...,...,...,...,...,...,...
183236,"Southwest not inspecting all CFM engines, sour...",southwest airlines co,NEW YORK (Reuters) - Southwest Airlines Co is ...,2018-04-19 01:22:00+03:00,None,"[LUV, southwest airlines company, 0.0]"
183253,The Rock’s Rampage Storms to No. 1 at Box Office,comscore inc.,The Rock is at it again.\nThe action-movie sta...,2018-04-15 21:48:00+03:00,"[SCOR, comscore inc, 0.0]",None
183291,Proxy adviser ISS backs call for gun safety re...,sturm ruger & co,"April 25, 2018 / 9:17 PM / in a day Proxy advi...",2018-04-26 00:16:00+03:00,None,"[RGR, sturm ruger & company, 0.0]"
183302,HSBC names veteran banker head of Asia belt an...,hsbc holdings plc,HONG KONG (Reuters) - HSBC Holdings Plc said o...,2018-04-09 14:51:00+03:00,None,"[HSBC, hsbc holdings plc, 0.0]"


In [146]:
timedeltas = {
    '+1day': pd.DateOffset(days=1),
    '+1week': pd.DateOffset(weeks=1),
    '+1month': pd.DateOffset(months=1),
    '+3month': pd.DateOffset(months=3),
    '+6month': pd.DateOffset(months=6)
}
for key,val in timedeltas.items():
    transformed_df[key] = transformed_df['published'] + val
transformed_df.head()

,title,entity,text,published,nasdaq_label,nyse_label,+1day,+1week,+1month,+3month,+6month
49,West Virginia sues Equifax over data breach,equifax inc,April 12 (Reuters) - West Virginia’s attorney ...,2018-04-13 03:29:00+03:00,None,"[EFX, equifax inc, 0.0]",2018-04-14 03:29:00+03:00,2018-04-20 03:29:00+03:00,2018-05-13 03:29:00+03:00,2018-07-13 03:29:00+03:00,2018-10-13 03:29:00+03:00
81,BRIEF-Emerald Expositions Says Board Elected L...,emerald expositions events inc,April 6 (Reuters) - Emerald Expositions Events...,2018-04-07 04:41:00+03:00,None,"[EEX, emerald expositions events inc, 0.0]",2018-04-08 04:41:00+03:00,2018-04-14 04:41:00+03:00,2018-05-07 04:41:00+03:00,2018-07-07 04:41:00+03:00,2018-10-07 04:41:00+03:00
108,New Residential Announces First Quarter 2018 R...,new residential investment corp.,NEW YORK--(BUSINESS WIRE)-- New Residential In...,2018-04-27 14:58:00+03:00,None,"[NRZ, new residential investment corp, 0.0]",2018-04-28 14:58:00+03:00,2018-05-04 14:58:00+03:00,2018-05-27 14:58:00+03:00,2018-07-27 14:58:00+03:00,2018-10-27 14:58:00+03:00
128,Bill Ackman’s Pershing Square Faces Wave of In...,blackstone group lp,"William Ackman, the famed shareholder activist...",2018-04-05 13:30:00+03:00,None,"[BX, the blackstone group lp, 0.0]",2018-04-06 13:30:00+03:00,2018-04-12 13:30:00+03:00,2018-05-05 13:30:00+03:00,2018-07-05 13:30:00+03:00,2018-10-05 13:30:00+03:00
130,Bill Ackman’s Pershing Square Faces Wave of In...,jpmorgan chase & co.,"William Ackman, the famed shareholder activist...",2018-04-05 13:30:00+03:00,None,"[JPM, jp morgan chase & co, 0.1]",2018-04-06 13:30:00+03:00,2018-04-12 13:30:00+03:00,2018-05-05 13:30:00+03:00,2018-07-05 13:30:00+03:00,2018-10-05 13:30:00+03:00


In [147]:
   
transformed_df[
    (transformed_df['nyse_label'].notna()) ^ (transformed_df['nasdaq_label'].notna())
].shape[0] / transformed_df.shape[0]
# 99,4 % of records have single label

0.9942461641094062

In [149]:
label_conflict = transformed_df[
    (transformed_df['nyse_label'].notna()) & (transformed_df['nasdaq_label'].notna())
]
label_conflict = label_conflict[
    label_conflict.apply(
        lambda row: row.loc['nasdaq_label'][0] != row.loc['nyse_label'][0],
        axis=1
    )
]
label_conflict

,title,entity,text,published,nasdaq_label,nyse_label,+1day,+1week,+1month,+3month,+6month
1078,Comcast internet subscriber growth tops video ...,comcast corp,April 25 (Reuters) - Comcast Corp posted quart...,2018-04-25 18:20:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-26 18:20:00+03:00,2018-05-02 18:20:00+03:00,2018-05-25 18:20:00+03:00,2018-07-25 18:20:00+03:00,2018-10-25 18:20:00+03:00
1237,BRIEF-Comcast And Charter Announce Mobile Oper...,comcast corp,April 20 (Reuters) - Charter Communications In...,2018-04-21 03:59:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-22 03:59:00+03:00,2018-04-28 03:59:00+03:00,2018-05-21 03:59:00+03:00,2018-07-21 03:59:00+03:00,2018-10-21 03:59:00+03:00
1345,First BanCorp to Announce 1Q 2018 Results on A...,first bancorp,"SAN JUAN, Puerto Rico--(BUSINESS WIRE)-- First...",2018-04-17 01:00:00+03:00,"[FNLC, first bancorp inc, 0.0]","[FBP, first bancorp, 0.0]",2018-04-18 01:00:00+03:00,2018-04-24 01:00:00+03:00,2018-05-17 01:00:00+03:00,2018-07-17 01:00:00+03:00,2018-10-17 01:00:00+03:00
4027,BRIEF-Republic First Bancorp Q1 Earnings Per S...,first bancorp inc,"April 23, 2018 / 2:15 PM / Updated 22 minutes ...",2018-04-23 17:15:00+03:00,"[FNLC, first bancorp inc, 0.0]","[FBP, first bancorp, 0.0]",2018-04-24 17:15:00+03:00,2018-04-30 17:15:00+03:00,2018-05-23 17:15:00+03:00,2018-07-23 17:15:00+03:00,2018-10-23 17:15:00+03:00
5224,Comcast will add Netflix to cable bundles in U.S.,comcast corp,"April 13, 2018 / 1:07 PM / Updated 11 minutes ...",2018-04-13 16:06:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-14 16:06:00+03:00,2018-04-20 16:06:00+03:00,2018-05-13 16:06:00+03:00,2018-07-13 16:06:00+03:00,2018-10-13 16:06:00+03:00
...,...,...,...,...,...,...,...,...,...,...,...
168516,Economist tells U.S. judge AT&T-Time Warner de...,comcast corp,"WASHINGTON, April 11 (Reuters) - The U.S. Just...",2018-04-12 02:48:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-13 02:48:00+03:00,2018-04-19 02:48:00+03:00,2018-05-12 02:48:00+03:00,2018-07-12 02:48:00+03:00,2018-10-12 02:48:00+03:00
176772,Economist tells U.S. judge AT&T-Time Warner de...,comcast corp,"April 11, 2018 / 6:55 PM / Updated 6 minutes a...",2018-04-11 21:54:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-12 21:54:00+03:00,2018-04-18 21:54:00+03:00,2018-05-11 21:54:00+03:00,2018-07-11 21:54:00+03:00,2018-10-11 21:54:00+03:00
177257,Economist tells U.S. judge AT&T-Time Warner de...,comcast corp,"April 11, 2018 / 6:55 PM / Updated 2 hours ago...",2018-04-12 02:24:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-13 02:24:00+03:00,2018-04-19 02:24:00+03:00,2018-05-12 02:24:00+03:00,2018-07-12 02:24:00+03:00,2018-10-12 02:24:00+03:00
179325,"Reuters Lifestyle & Entertainment, April 13, 1...",comcast corp,Contact info for Reuters Entertainment & Lifes...,2018-04-13 23:01:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-14 23:01:00+03:00,2018-04-20 23:01:00+03:00,2018-05-13 23:01:00+03:00,2018-07-13 23:01:00+03:00,2018-10-13 23:01:00+03:00


In [150]:
label_conflict['nasdaq_label'].apply(lambda e: e[0]).value_counts()

CMCSA    51
FNLC      7
CYHHZ     3
OCSL      2
PSEC      1
AINV      1
Name: nasdaq_label, dtype: int64

In [151]:
label_conflict[
    label_conflict['nasdaq_label'] == label_conflict['nyse_label']
]

,title,entity,text,published,nasdaq_label,nyse_label,+1day,+1week,+1month,+3month,+6month


In [152]:
label_conflict[
    label_conflict['nasdaq_label'] != label_conflict['nyse_label']
]

,title,entity,text,published,nasdaq_label,nyse_label,+1day,+1week,+1month,+3month,+6month
1078,Comcast internet subscriber growth tops video ...,comcast corp,April 25 (Reuters) - Comcast Corp posted quart...,2018-04-25 18:20:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-26 18:20:00+03:00,2018-05-02 18:20:00+03:00,2018-05-25 18:20:00+03:00,2018-07-25 18:20:00+03:00,2018-10-25 18:20:00+03:00
1237,BRIEF-Comcast And Charter Announce Mobile Oper...,comcast corp,April 20 (Reuters) - Charter Communications In...,2018-04-21 03:59:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-22 03:59:00+03:00,2018-04-28 03:59:00+03:00,2018-05-21 03:59:00+03:00,2018-07-21 03:59:00+03:00,2018-10-21 03:59:00+03:00
1345,First BanCorp to Announce 1Q 2018 Results on A...,first bancorp,"SAN JUAN, Puerto Rico--(BUSINESS WIRE)-- First...",2018-04-17 01:00:00+03:00,"[FNLC, first bancorp inc, 0.0]","[FBP, first bancorp, 0.0]",2018-04-18 01:00:00+03:00,2018-04-24 01:00:00+03:00,2018-05-17 01:00:00+03:00,2018-07-17 01:00:00+03:00,2018-10-17 01:00:00+03:00
4027,BRIEF-Republic First Bancorp Q1 Earnings Per S...,first bancorp inc,"April 23, 2018 / 2:15 PM / Updated 22 minutes ...",2018-04-23 17:15:00+03:00,"[FNLC, first bancorp inc, 0.0]","[FBP, first bancorp, 0.0]",2018-04-24 17:15:00+03:00,2018-04-30 17:15:00+03:00,2018-05-23 17:15:00+03:00,2018-07-23 17:15:00+03:00,2018-10-23 17:15:00+03:00
5224,Comcast will add Netflix to cable bundles in U.S.,comcast corp,"April 13, 2018 / 1:07 PM / Updated 11 minutes ...",2018-04-13 16:06:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-14 16:06:00+03:00,2018-04-20 16:06:00+03:00,2018-05-13 16:06:00+03:00,2018-07-13 16:06:00+03:00,2018-10-13 16:06:00+03:00
...,...,...,...,...,...,...,...,...,...,...,...
168516,Economist tells U.S. judge AT&T-Time Warner de...,comcast corp,"WASHINGTON, April 11 (Reuters) - The U.S. Just...",2018-04-12 02:48:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-13 02:48:00+03:00,2018-04-19 02:48:00+03:00,2018-05-12 02:48:00+03:00,2018-07-12 02:48:00+03:00,2018-10-12 02:48:00+03:00
176772,Economist tells U.S. judge AT&T-Time Warner de...,comcast corp,"April 11, 2018 / 6:55 PM / Updated 6 minutes a...",2018-04-11 21:54:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-12 21:54:00+03:00,2018-04-18 21:54:00+03:00,2018-05-11 21:54:00+03:00,2018-07-11 21:54:00+03:00,2018-10-11 21:54:00+03:00
177257,Economist tells U.S. judge AT&T-Time Warner de...,comcast corp,"April 11, 2018 / 6:55 PM / Updated 2 hours ago...",2018-04-12 02:24:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-13 02:24:00+03:00,2018-04-19 02:24:00+03:00,2018-05-12 02:24:00+03:00,2018-07-12 02:24:00+03:00,2018-10-12 02:24:00+03:00
179325,"Reuters Lifestyle & Entertainment, April 13, 1...",comcast corp,Contact info for Reuters Entertainment & Lifes...,2018-04-13 23:01:00+03:00,"[CMCSA, comcast corp a, 0.0]","[CCZ, comcast corp, 0.0]",2018-04-14 23:01:00+03:00,2018-04-20 23:01:00+03:00,2018-05-13 23:01:00+03:00,2018-07-13 23:01:00+03:00,2018-10-13 23:01:00+03:00


In [153]:
label_conflict[
    label_conflict.apply(
        lambda row: row.loc['entity'] == row.loc['nasdaq_label'][1] or row.loc['entity'] == row.loc['nyse_label'][1],
        axis=1
    )
].shape[0] / label_conflict.shape[0]

0.9692307692307692

In [155]:
label_conflict[
    label_conflict.apply(
        lambda row: (row.loc['entity'] == row.loc['nasdaq_label'][1]) ^ (row.loc['entity'] == row.loc['nyse_label'][1]),
        axis=1
    )
].shape[0] / label_conflict.shape[0]
# 96% entities match exactly one label (by simple string comparison)

0.9692307692307692

In [156]:
label_conflict[
    label_conflict.apply(
        lambda row: row.loc['entity'] != row.loc['nasdaq_label'][1] and row.loc['entity'] != row.loc['nyse_label'][1],
        axis=1
    )
]

,title,entity,text,published,nasdaq_label,nyse_label,+1day,+1week,+1month,+3month,+6month
146675,Oaktree Specialty Lending Corporation Schedule...,oaktree specialty lending corporation,"LOS ANGELES, April 12, 2018 (GLOBE NEWSWIRE) -...",2018-04-12 16:00:00+03:00,"[OCSL, oaktree specialty lending corp, 0.0]","[OSLE, oaktree specialty lending corp, 0.0]",2018-04-13 16:00:00+03:00,2018-04-19 16:00:00+03:00,2018-05-12 16:00:00+03:00,2018-07-12 16:00:00+03:00,2018-10-12 16:00:00+03:00
146677,Oaktree Specialty Lending Corporation Schedule...,oaktree specialty lending,"LOS ANGELES, April 12, 2018 (GLOBE NEWSWIRE) -...",2018-04-12 16:00:00+03:00,"[OCSL, oaktree specialty lending corp, 0.0]","[OSLE, oaktree specialty lending corp, 0.0]",2018-04-13 16:00:00+03:00,2018-04-19 16:00:00+03:00,2018-05-12 16:00:00+03:00,2018-07-12 16:00:00+03:00,2018-10-12 16:00:00+03:00


## Observation from "nasdaq vs. nyse" notebook

99% of nasdaq labels were not present in nyse, i.e. considered unique, compared to only 80% of nyse labels

Therefore "nasdaq" will be used first for identifying companies by label, after that "nyse" in case nasdaq is not found

In [112]:
timedeltas

{'+1day': <DateOffset: days=1>,
 '+1week': <DateOffset: weeks=1>,
 '+1month': <DateOffset: months=1>,
 '+3month': <DateOffset: months=3>,
 '+6month': <DateOffset: months=6>}

# Forgot to consider timezone differences for written articles

In [253]:
import datetime

def get_stock_price_from_stock_df(date, stock_df, column_name):
    result = stock_df[stock_df['date'] <= date]
    return None if result.shape[0] == 0 else result.iloc[0].loc[column_name]

def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        if helper.calls % 1000 == 0:
            print(f'calls: {helper.calls}')
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__

    return helper
# Really slow function
@call_counter
def lookup_stock_prices(row, column_name='close'):
    article_published_date = row.loc['published']
    nasdaq_label = row.loc['nasdaq_label']
    nyse_label = row.loc['nyse_label']
    stock_symbol = nyse_label[0] if nasdaq_label is None else nasdaq_label[0]
    #print(f'Checking out dataframe for symbol "{stock_symbol}"')
    file = path.join(cwd, f'../data/stock-data/full_history/{stock_symbol}.csv')
    result = pd.read_csv(file)
    result['date'] = result.date.apply(lambda x: pd.Timestamp(x).date())
    for key,val in timedeltas.items():
        row[f'stock_{key}'] = None
    row['stock_published'] = get_stock_price_from_stock_df(
        row.loc['published'].date(),
        result,
        column_name
    )
    for key,val in timedeltas.items():
        date_index = row.loc[key].date()
        row[f'stock_{key}'] = get_stock_price_from_stock_df(date_index, result, column_name)
    return row
    
lookup_stock_prices(transformed_df.iloc[0])

<ipython-input-253-239e145de7a8>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[f'stock_{key}'] = None
<ipython-input-253-239e145de7a8>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['stock_published'] = get_stock_price_from_stock_df(
<ipython-input-253-239e145de7a8>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[f'stock_{key}'] = get_stock_price_from_stock_df(date_index, result, column_name)


title                    West Virginia sues Equifax over data breach
entity                                                   equifax inc
text               April 12 (Reuters) - West Virginia’s attorney ...
published                                  2018-04-13 03:29:00+03:00
nasdaq_label                                                    None
nyse_label                                   [EFX, equifax inc, 0.0]
+1day                                      2018-04-14 03:29:00+03:00
+1week                                     2018-04-20 03:29:00+03:00
+1month                                    2018-05-13 03:29:00+03:00
+3month                                    2018-07-13 03:29:00+03:00
+6month                                    2018-10-13 03:29:00+03:00
stock_published                                                  116
stock_+1day                                                      116
stock_+1week                                                  118.88
stock_+1month                     

In [223]:
transformed_df['entity'].unique().shape[0] / transformed_df.shape[0]

0.3720813875917278

In [224]:
test = transformed_df[transformed_df['entity'] != 'thomson reuters']
test['entity'].unique().shape[0] / test.shape[0]

0.3720813875917278

In [225]:
transformed_df['entity'].value_counts()

facebook inc                   259
deutsche bank                  233
morgan stanley                 188
amazon.com inc                 163
tesla inc                      117
                              ... 
cal industries                   1
pivot pharmaceuticals inc        1
houston wire & cable co          1
cna financial corp               1
gardner denver holdings inc      1
Name: entity, Length: 4462, dtype: int64

In [226]:
transformed_df['published'].apply(lambda x: x.tz).value_counts()

pytz.FixedOffset(180)    11989
pytz.FixedOffset(120)        3
Name: published, dtype: int64

In [237]:
vc = transformed_df.apply(lambda row: (row.loc['entity'], row.loc['published']), axis=1).value_counts()
vc[vc > 1]
# Combination of entity name and published article date is not unique

(morningstar , inc, 2018-04-25 03:00:00+03:00)                   10
(morningstar , inc, 2018-04-26 03:00:00+03:00)                    7
(tyson foods, 2018-04-02 18:24:00+03:00)                          4
(wta international, 2018-04-12 04:20:00+03:00)                    4
(royal bank of scotland group plc, 2018-04-27 14:13:00+03:00)     4
                                                                 ..
(lasalle hotel properties inc, 2018-04-02 15:24:00+03:00)         2
(tesla inc, 2018-04-26 22:00:00+03:00)                            2
(conocophillips, 2018-04-02 23:53:00+03:00)                       2
(amazon.com inc, 2018-04-28 00:39:00+03:00)                       2
(bb&t corp, 2018-04-19 13:12:00+03:00)                            2
Length: 430, dtype: int64

In [248]:
def build_file_index_dict(t_df):
    result = {}
    for index, data in t_df.iterrows():
        nyse_symbol = data.loc['nyse_label']
        nasdaq_symbol = data.loc['nasdaq_label']
        symbol = nyse_symbol if nasdaq_symbol is None else nasdaq_symbol
        stock_symbol = symbol[0]
        if stock_symbol not in result:
            result[stock_symbol] = []
        result[stock_symbol].append(index)
    return result
file_lookup_dict = build_file_index_dict(transformed_df)


In [263]:
@call_counter
def process_row_ids(row_ids, df, stock_df):
    for row_id in row_ids:
        df.loc[row_id, 'stock_published'] = get_stock_price_from_stock_df(
            df.loc[row_id,'published'].date(),
            stock_df,
            'close'
        )
        for key,val in timedeltas.items():
            df.loc[row_id, f'stock_{key}'] = get_stock_price_from_stock_df(
                df.loc[row_id, key].date(),
                stock_df,
                'close'
            )

transformed_df['stock_published'] = None
for key,val in timedeltas.items():
    transformed_df[f'stock_{key}'] = None
for stock_symbol, row_ids in file_lookup_dict.items():
    file = path.join(cwd, f'../data/stock-data/full_history/{stock_symbol}.csv')
    stock_df = pd.read_csv(file)
    stock_df['date'] = stock_df['date'].apply(lambda x: pd.Timestamp(x).date())
    process_row_ids(row_ids, transformed_df, stock_df)

transformed_df.head()

calls: 1000
calls: 2000


,title,entity,text,published,nasdaq_label,nyse_label,+1day,+1week,+1month,+3month,+6month,stock_published,stock_+1day,stock_+1week,stock_+1month,stock_+3month,stock_+6month
49,West Virginia sues Equifax over data breach,equifax inc,April 12 (Reuters) - West Virginia’s attorney ...,2018-04-13 03:29:00+03:00,None,"[EFX, equifax inc, 0.0]",2018-04-14 03:29:00+03:00,2018-04-20 03:29:00+03:00,2018-05-13 03:29:00+03:00,2018-07-13 03:29:00+03:00,2018-10-13 03:29:00+03:00,116,116,118.88,114.72,126.99,123.05
81,BRIEF-Emerald Expositions Says Board Elected L...,emerald expositions events inc,April 6 (Reuters) - Emerald Expositions Events...,2018-04-07 04:41:00+03:00,None,"[EEX, emerald expositions events inc, 0.0]",2018-04-08 04:41:00+03:00,2018-04-14 04:41:00+03:00,2018-05-07 04:41:00+03:00,2018-07-07 04:41:00+03:00,2018-10-07 04:41:00+03:00,19.67,19.67,19.56,19.54,20.62,15.84
108,New Residential Announces First Quarter 2018 R...,new residential investment corp.,NEW YORK--(BUSINESS WIRE)-- New Residential In...,2018-04-27 14:58:00+03:00,None,"[NRZ, new residential investment corp, 0.0]",2018-04-28 14:58:00+03:00,2018-05-04 14:58:00+03:00,2018-05-27 14:58:00+03:00,2018-07-27 14:58:00+03:00,2018-10-27 14:58:00+03:00,17.35,17.35,17.94,18.1,17.78,17.46
128,Bill Ackman’s Pershing Square Faces Wave of In...,blackstone group lp,"William Ackman, the famed shareholder activist...",2018-04-05 13:30:00+03:00,None,"[BX, the blackstone group lp, 0.0]",2018-04-06 13:30:00+03:00,2018-04-12 13:30:00+03:00,2018-05-05 13:30:00+03:00,2018-07-05 13:30:00+03:00,2018-10-05 13:30:00+03:00,31.3,31.01,31.13,31.81,35.13,37.34
130,Bill Ackman’s Pershing Square Faces Wave of In...,jpmorgan chase & co.,"William Ackman, the famed shareholder activist...",2018-04-05 13:30:00+03:00,None,"[JPM, jp morgan chase & co, 0.1]",2018-04-06 13:30:00+03:00,2018-04-12 13:30:00+03:00,2018-05-05 13:30:00+03:00,2018-07-05 13:30:00+03:00,2018-10-05 13:30:00+03:00,111.88,109.09,113.37,108.43,103.72,114.62


In [264]:
transformed_df.shape

(11992, 17)

# Algorithm 3 - Combining into final dataset

Requires to load `pd.DataFrame` from `../data/labels_lookup.json` file 1st - loaded as `translation_df`

#### Functions/vars from above
* `remove_published`
* `process_row_ids`
* `timedeltas` dictionary

#### Dependencies
* os
* pandas

In [266]:
def process_file(fp):
    print(f'Processing file: "{fp}"')
    df = pd.read_json(fp)
    print(f'[{fp}]: sanitizing entities, removing detected publishers')
    df['entities_blacklist'] = df['thread.site_full'].apply(lambda entry: entry.split('.')[-2])
    df = df.apply(remove_published, axis=1)
    df['entities.organizations'] = df['entities.organizations'].apply(
        lambda l: list(map(lambda item: item['name'], l))
    )
    df = df.drop(columns=['thread.site_full','entities_blacklist'])
    print(f'[{fp}]: Transforming into 2NF')
    transformed_df = pd.DataFrame(
        [(row[1], entity, row[3], row[4]) for row in df.itertuples() for entity in row[2]],
        columns = df.columns
    ).rename(columns={'entities.organizations': 'entity'})
    print(f'[{fp}] JOINing with "translation_df" to get precomputed stock_symbols')
    transformed_df['nasdaq_label'] = transformed_df['entity'].apply(
        lambda e: translation_df.loc[e,'nasdaq_label'] if e in translation_df.index else None
    )
    transformed_df['nyse_label'] = transformed_df['entity'].apply(
        lambda e: translation_df.loc[e,'nyse_label'] if e in translation_df.index else None
    )
    print(f'[{fp}] Current size: {transformed_df.shape[0]}, dropping unnecessary records')
    transformed_df = transformed_df[
        (transformed_df['nasdaq_label'].notna()) | (transformed_df['nyse_label']).notna()
    ]
    print(f'[{fp}] Reduced size: {transformed_df.shape[0]}')
    return transformed_df

input_files = [ path.join(input_dir, file) for file in listdir(input_dir) ]
print('Combining loaded dataframes')
df = pd.concat(map(process_file, input_files), sort=False).reset_index().drop(columns=['index'])
print('Initializing label columns')
df['published'] = df['published'].apply(pd.Timestamp)
df['stock_published'] = None
for key,val in timedeltas.items():
    df[key] = df['published'] + val
    df[f'stock_{key}'] = None
f_index_dict = build_file_index_dict(df)
print(f'Initialized dictionary with {len(f_index_dict)} entries')
for stock_symbol, row_ids in f_index_dict.items():
    file = path.join(cwd, f'../data/stock-data/full_history/{stock_symbol}.csv')
    stock_df = pd.read_csv(file)
    stock_df['date'] = stock_df['date'].apply(lambda x: pd.Timestamp(x).date())
    process_row_ids(row_ids, df, stock_df)
df.head()

Combining loaded dataframes
Processing file: "/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-05.json"
[/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-05.json]: sanitizing entities, removing detected publishers
[/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-05.json]: Transforming into 2NF
[/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-05.json] JOINing with "translation_df" to get precomputed stock_symbols
[/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-05.json] Current size: 182717, dropping unnecessary records
[/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-05.json] Reduced size: 11858
Processing file: "/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-04.json"
[/Users/melkos01/Documents/aitalents-investoid/notebooks/../data/prepared/2018-04.json]: sanitizing entities, remo

,title,entity,text,published,nasdaq_label,nyse_label,stock_published,+1day,stock_+1day,+1week,stock_+1week,+1month,stock_+1month,+3month,stock_+3month,+6month,stock_+6month
0,ANZ Bank New Zealand sells life insurance busi...,cigna corp,May 30 (Reuters) - ANZ Bank New Zealand on Wed...,2018-05-30 02:42:00+03:00,None,"[CI, cigna corp, 0.0]",173.32,2018-05-31 02:42:00+03:00,169.37,2018-06-06 02:42:00+03:00,174.93,2018-06-30 02:42:00+03:00,169.95,2018-08-30 02:42:00+03:00,189.8,2018-11-30 02:42:00+03:00,213.13
1,Australia's Westpac cleared of rate rigging bu...,westpac banking corp,SYDNEY (Reuters) - Australia’s Westpac Banking...,2018-05-24 08:14:00+03:00,None,"[WBK, westpac banking corp, 0.0]",21.46,2018-05-25 08:14:00+03:00,21.31,2018-05-31 08:14:00+03:00,21.14,2018-06-24 08:14:00+03:00,21.8,2018-08-24 08:14:00+03:00,20.11,2018-11-24 08:14:00+03:00,19.07
2,"Clear Channel Outdoor Holdings, Inc. Reports R...","clear channel outdoor holdings, inc.",SAN ANTONIO--(BUSINESS WIRE)-- Clear Channel O...,2018-05-22 14:15:00+03:00,None,"[CCO, clear channel outdoor holdings, 0.0]",4.5,2018-05-23 14:15:00+03:00,4.45,2018-05-29 14:15:00+03:00,4.4,2018-06-22 14:15:00+03:00,4.65,2018-08-22 14:15:00+03:00,4.7,2018-11-22 14:15:00+03:00,5.86
3,"Clear Channel Outdoor Holdings, Inc. Reports R...","clear channel outdoor holdings, inc",SAN ANTONIO--(BUSINESS WIRE)-- Clear Channel O...,2018-05-22 14:15:00+03:00,None,"[CCO, clear channel outdoor holdings, 0.0]",4.5,2018-05-23 14:15:00+03:00,4.45,2018-05-29 14:15:00+03:00,4.4,2018-06-22 14:15:00+03:00,4.65,2018-08-22 14:15:00+03:00,4.7,2018-11-22 14:15:00+03:00,5.86
4,BRIEF-Pembina Pipeline Corp Reports Q1 EPS $0.59,brief-pembina pipeline corp,May 3 (Reuters) - Pembina Pipeline Corp:\n* OR...,2018-05-04 01:43:00+03:00,None,"[PBA, pembina pipeline cor, 0.0]",33.94,2018-05-05 01:43:00+03:00,33.94,2018-05-11 01:43:00+03:00,34.65,2018-06-04 01:43:00+03:00,34.46,2018-08-04 01:43:00+03:00,36.59,2018-11-04 01:43:00+03:00,33.07


In [270]:
df.to_json(path.join(cwd, '../training.json'))

In [271]:
df.shape

(56101, 17)